# Lab Assignment

**Exercise Description** <br>
In this exercise, you are required to implement an emergency braking system on a vehicle in 
the CARLA simulator using a radar sensor. The vehicle must be able to: 
- Detect obstacles or vehicles ahead using the radar. 
- Calculate the Time to Collision (TTC) based on the relative speed and distance of the 
obstacle. 
- Activate emergency braking if the TTC falls below a predefined threshold to avoid a 
collision.

**Requirements** <br>
1. Radar Sensor Configuration: 
    - Add a radar sensor to the vehicle in the simulator. 
    - Configure the sensor to collect relevant data (distance, relative speed, angle). 
2. Radar Data Processing: 
    - Write an algorithm to process radar data in real-time. 
    - Identify relevant obstacles ahead of the vehicle. 
    - Calculate the TTC for each obstacle. 
3. Implementation of the Braking System: 
    - Define a TTC threshold to activate braking. 
    - Implement a control that applies emergency braking when necessary. 
4. Testing and Validation: 
    - Create test scenarios in the simulator to verify the system's functionality. 
    - Document the results, including cases where braking successfully prevents a collision

In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

In [ ]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

In [ ]:
target_vehicle = spawn_vehicle()

# 1. Add lidar radar sensor to the vehicle
lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '50')
lidar_bp.set_attribute('rotation_frequency', '10')
lidar_bp.set_attribute('channels', '32')
lidar_bp.set_attribute('points_per_second', '56000')
lidar_transform = carla.Transform(carla.Location(x=0, z=2.5))
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=target_vehicle)

# 2. sensor has to collect relevant data (distance, relative speed, angle)
# 3. breaking system implemetation
TTC_threshold = 2.0

def get_TTC(distance, relative_speed):
    if relative_speed == 0:
        return 1000
    return distance / relative_speed

def brake():
    target_vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))

def callback_lidar(data):
    points = np.frombuffer(data.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (int(points.shape[0] / 3), 3))
    for point in points:
        distance = math.sqrt(point[0]**2 + point[1]**2 + point[2]**2)
        relative_speed = target_vehicle.get_velocity().x
        TTC = get_TTC(distance, relative_speed)
        if TTC < TTC_threshold:
            brake()

lidar.listen(callback_lidar)